# Lab 5 - Apache Spark - wprowadzenie.

Apache Spark jest silnikiem do przetwarzania dancych na dużą skalę, pozwalający na wykonywanie operacji w sposób zrównoleglony i rozproszony. Spark dostarcza API dla języków java, Scala, Python oraz R do przetwarzania grafów obliczeń. Spark składa się z wielu narzędzi takich jak:
* Resilient Distributed Datasets (RDD) - niskopoziomowy typ zbioru danych Spark, na którym opierają się struktury danych na wyższych poziomach abstrakcji,
* Spark SQL - Spark Dataset, Spark DataFrame,
* Pandas API on Spark - API pozwalające na wykorzystanie biblioteki pandas w sposób zrównoleglony na klastrze Spark,
* Structured Streaming - zestaw narzędzi do przetwarzania strumieniowego,
* MLlib - moduł wspierający wykorzystanie Machine Learning z użyciem typów danych Spark oraz klastrów Spark,
* GraphX - przetwarzanie grafów,
* SparkR - API w języku R do pracy w środowisku Spark,
* PySpark - API Python do pracy w środowisku Spark,
* Spark SQL CLI - przetwarzanie danych z użyciem Spark SQL z poziomu wiersza poleceń.

## 1. Instalacja PySpark i przygotowanie środowiska pracy.

W trakcie zajęc z racji dotychczasowego środowiska pracy (Python) będzie wykorzystywane API Pythona, które dostarcza Spark.

**Dokumentacja Spark Python API:** https://spark.apache.org/docs/3.5.3/api/python/index.html

> Oficjalna dokumentacja alternatywnych sposobów instalacji oraz zależności modułu PySpark: https://spark.apache.org/docs/latest/api/python/getting_started/install.html


In [ ]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

> Przed uruchomieniem poniższej komórki należy zainstalować moduł `pyspark` do środowiska wirtualnego Pythona

> Oficjana dokumentacja konfiguracji Sparka, również parametrów wywołania: https://spark.apache.org/docs/3.5.3/configuration.html

Aby możliwe było wysłanie zadań do wykonania z wykorzystaniem klastra Spark (lokalnego lub rozproszonego) musimy uzyskać najpierw referencję do obiektu typu SparkSession. Tworząc obiekt musimy zdefiniować nazwę aplikacji, którą otrzyma ta sesja oraz możemy zdefiniować dodatkowe parametry. Przykład poniżej.

In [ ]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Podobnie jak w przypadku biblioteki Dask możemy śledzić pracę klastra poprzez przeglądarkę. Adres to http://\<host\>:4040. Wykorzystując dockera wymagane jest stworzenie odpowiedniego mapowania portu oraz w naszym przypadków również zmiany domyślnego hosta, który zapewne będzie skróconą wersją hasha kontenera. Skoro port jest zmapowany to znaczy, że będzie dostępny na hoście lokalnym. Finalnie więc adres dla węzła master powinien być dostępny pod adresem `http://localhost:4040`.

## 2. Spark Resilient Distributed Datasets (RDD).

Na wysokim poziomie każda aplikacja Spark składa się z **programu sterownika (ang. driver program)**, który uruchamia główną funkcję użytkownika i wykonuje różne operacje równoległe na klastrze. 

Główną abstrakcją zapewnianą przez Spark jest **odporny rozproszony zbiór danych (Resilient Distributed Datasets - RDD)**, który jest zbiorem elementów podzielonych na węzły klastra, które mogą być obsługiwane równolegle. RDD mogą być również utrwalane w pamięci klastra, umożliwiając jego ponowne wykorzystanie w operacjach równoległych. Wreszcie, RDD automatycznie odzyskują sprawność po awarii węzła.

Wykorzystanie API RDD nie jest rekomendowane w wielu przypadkach, gdyż wymaga dobrej znajomości niskopoziomowego API Sparka i ręcznej optymalizacji operacji. Zazwyczaj lepszym pomysłem będzie wykorzystanie Spark Dataset oraz Spark DataFrame.

Kilka przykładów wykorzystania tego API zostanie jednak tutaj przedstawionych.

> Poradnik programisty dla RDD znajduje się pod linkiem: https://spark.apache.org/docs/3.5.3/rdd-programming-guide.html

In [ ]:
# lista wartości zostaje podzielona na partycje i rozproszona na wszystkie dostępne węzły
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.SparkContext.parallelize.html
rdd = spark.sparkContext.parallelize(list(range(20)))

# rdd w formie rozproszonej zostaje scalone w listę zawierającą wszystkie elementy RDD
# np. za pomocą funkcji collect()
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.RDD.collect.html

rddCollect = rdd.collect()
display(type(rddCollect))
print(f"Liczba partycji: {rdd.getNumPartitions()}")
print(f"Pierwszy element: {rdd.first()}")
print(rddCollect)

In [ ]:
# obiekt RDD może przechowywać dane z różnych źródeł, które są zgodne z systemem plików Apache Hadoop
# np. Amazon S3, Cassandra, HDFS, HBase i inne

# możemy dla uniknięcia potrzeby każdorazowego odwoływania się do kontekstu poprzez spark.sparkContext zapisać sobie to w zmiennej pomocniczej
sc = spark.sparkContext
# tutaj wczytamy do RDD plik tekstowy
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

Więcej informacji odnośnie obsługi plików w środowisku Spark można znaleźć m.in. tu: https://spark.apache.org/docs/3.5.3/rdd-programming-guide.html#external-datasets

In [ ]:
display(pan_tadeusz_file.getNumPartitions())
pan_tadeusz_file.take(10)

In [ ]:
# możemy zmienić liczbę automatycznie stworzonych partycji i ponownie rozproszyć je po węzłach
pan_tadeusz_file = pan_tadeusz_file.repartition(4)
pan_tadeusz_file.getNumPartitions()

# również metoda coalesce może posłużyć nam do zmiany ilości partycji dla obiektu RDD np. po zastosowaniu filtrowania, które
# znacznie zmniejsza wielkość pierwotnego obiektu RDD a co za tym idzie każdej partycji i dalsze obliczenia mogą nie być
# wykonywane zbyt efektywnie (zbyt mały rozmiar partycji)
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.RDD.coalesce.html
# główna różnica między repartition a coalesce jest taka, że ta pierwsza wykorzystuje mechanizm tasowania danych a ta druga może, ale nie
# musi go wykorzystywać gdyż możemy tym sterować za pomocą parametru wywołania tej metody

In [ ]:
# jedną z funkcji dostępnej w tym API jest możliwość wykonania funkcji na każdej z partycji
# minusem może być to, że funkcja foreachPartition zwraca typ None, więc wyniki należy przetworzyć w inny sposób

def count_words(iterator):
    words = sum([len(x.split()) for x in iterator])
    print(words)

pan_tadeusz_file.foreachPartition(count_words)

Przy wypisywaniu wartości z RDD trzeba również zwrócić uwagę na różnicę w działaniu tych metod w trybie pracy lokalnej (czyli tak jak w tym labie) oraz klastra. Efekty mogą być różne, więcej przeczytasz tu: https://spark.apache.org/docs/3.5.3/rdd-programming-guide.html#printing-elements-of-an-rdd

**RDD obsługują dwa rodzaje operacji: transformacje**, które tworzą nowy zbiór danych z istniejącego, oraz **akcje**, które zwracają wartość do programu sterownika po uruchomieniu obliczeń na zbiorze danych. Przykładowo, map jest transformacją, która przepuszcza każdy element zbioru danych przez funkcję i zwraca nowy RDD reprezentujący wyniki. Z drugiej strony, reduce jest akcją, która agreguje wszystkie elementy RDD przy użyciu pewnej funkcji i zwraca końcowy wynik do programu sterownika.

Wszystkie transformacje są wykonywane w sposób **leniwy** tzn, że obliczenia nie są wykonywane dopóki nie jest potrzebnych wynik. To podobnie jak w przypadku frameworka Dask umożliwia optymalizację obliczeń np. w przypadku gdy nie są potrzebne wyniki pośrednie z każdego węzła po zastosowaniu funkcji poprzez `map`, ale tylo wynik akcji `reduce`, więc nie ma potrzeby przesyłania całych pośrednich RDD do drivera.

**Lista wybranych transformacji dostępna jest tu:** https://spark.apache.org/docs/3.5.3/rdd-programming-guide.html#transformations

**Lista wybranych akcji tu:** https://spark.apache.org/docs/3.5.3/rdd-programming-guide.html#actions

W uzasadnionych przypadkach można również przyspieszyć obliczenia poprzez utrwalenie danych w pamięci lub pamięci podręcznej poprzez metody `persist` lub `cache` na obiekcie RDD.

**Kilka przykładów transformacji i akcji**

In [ ]:
# funkcje map oraz reduce

# możemy również wykonać operację w inny sposób, tym raze mapując funkcję na każdy element obiektu RDD
# zwrócony zostanie obiekt RDD, na którym możemy wykonać kolejne operacje
pan_tadeusz_file.map(lambda s: len(s.split())).take(10)

# np. reduce - i tu nawiązanie do dość znanej techniki przetwarzania, MapReduce
# więcej: https://en.wikipedia.org/wiki/MapReduce
# oraz: https://wiadrodanych.pl/big-data/jak-dziala-mapreduce/
pan_tadeusz_file.map(lambda s: len(s.split())).reduce(lambda a, b: a + b)

In [ ]:
# lub tak - ten sam efekt
from operator import add
pan_tadeusz_file.map(lambda s: len(s.split())).reduce(add)

In [ ]:
# różnica między map() a flatMap() dla tego przypadku
display(pan_tadeusz_file.map(lambda s: s.split()).take(10))
pan_tadeusz_file.flatMap(lambda s: s.split()).take(10)

### Zadania

**Zadanie 1**  

Wykorzystując plik z treścią Pana Tadeusza zaprezentowany w przykładach policz ile jest linii, w których zawiera się słowo `Tadeusz`. Wykorzystaj akcje i transformacje RDD.

**Zadanie 2**  
Wykorzystując metodę `top()` dla obiektu RDD wyświetl 3 najdłuższe linie wczytane z pliku z książką Pan Tadeusz.

**Zadanie 3**  
Wykorzystując listę stopwords z adresu https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt wykorzystaj akcje i transformacje RDD i wygeneruj listę unikalnych słów z pliku z treścią Pana Tadeusza (sprawdź transformację countByValue()) pomijając powyższe słowa stop oraz wszelkie znaki przestankowe. Wynik zapisz do słownika, a następnie do pliku json o nazwie pan_tadeusz_bag_of_words.json.
Które słowo występuje w tym tekście najczęściej? Wyświetl je z wyników wygenerowanych powyżej.